<a href="https://colab.research.google.com/github/EMORI-SERVULO/Public/blob/master/redNueronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np


In [0]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def sigmoid_derivada(x):
    return sigmoid(x)*(1.0-sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_derivada(x):
    return 1.0 - x**2

In [0]:
class NeuralNetwork:

    def __init__(self, layers, activation='tanh'):
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_prime = sigmoid_derivada
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_prime = tanh_derivada

        # inicializo los pesos
        self.weights = []
        self.deltas = []
        # capas = [2,3,2]
        # rando de pesos varia entre (-1,1)
        # asigno valores aleatorios a capa de entrada y capa oculta
        for i in range(1, len(layers) - 1):
            r = 2*np.random.random((layers[i-1] + 1, layers[i] + 1)) -1
            self.weights.append(r)
        # asigno aleatorios a capa de salida
        r = 2*np.random.random( (layers[i] + 1, layers[i+1])) - 1
        self.weights.append(r)

    def fit(self, X, y, learning_rate=0.2, epochs=100000):
        # Agrego columna de unos a las entradas X
        # Con esto agregamos la unidad de Bias a la capa de entrada
        ones = np.atleast_2d(np.ones(X.shape[0]))
        X = np.concatenate((ones.T, X), axis=1)
        
        for k in range(epochs):
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            for l in range(len(self.weights)):
                    dot_value = np.dot(a[l], self.weights[l])
                    activation = self.activation(dot_value)
                    a.append(activation)
            # Calculo la diferencia en la capa de salida y el valor obtenido
            error = y[i] - a[-1]
            deltas = [error * self.activation_prime(a[-1])]
            
            # Empezamos en el segundo layer hasta el ultimo
            # (Una capa anterior a la de salida)
            for l in range(len(a) - 2, 0, -1): 
                deltas.append(deltas[-1].dot(self.weights[l].T)*self.activation_prime(a[l]))
            self.deltas.append(deltas)

            # invertir
            # [level3(output)->level2(hidden)]  => [level2(hidden)->level3(output)]
            deltas.reverse()

            # backpropagation
            # 1. Multiplcar los delta de salida con las activaciones de entrada 
            #    para obtener el gradiente del peso.
            # 2. actualizo el peso restandole un porcentaje del gradiente
            for i in range(len(self.weights)):
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta)

            if k % 10000 == 0: print('epochs:', k)

    def predict(self, x): 
        ones = np.atleast_2d(np.ones(x.shape[0]))
        a = np.concatenate((np.ones(1).T, np.array(x)), axis=0)
        for l in range(0, len(self.weights)):
            a = self.activation(np.dot(a, self.weights[l]))
        return a

    def print_weights(self):
        print("LISTADO PESOS DE CONEXIONES")
        for i in range(len(self.weights)):
            print(self.weights[i])

    def get_deltas(self):
        return self.deltas

In [16]:
# funcion XOR
nn = NeuralNetwork([2,2,1])
X = np.array([[0, 0],
            [0, 1],
            [1, 0],
            [1, 1]])
y = np.array([0, 1, 1, 0])
nn.fit(X, y,epochs=2500)
for e in X:
    print("Entrdas:",e,"Salidas:",nn.predict(e))

epochs: 0
Entrdas: [0 0] Salidas: [0.01459908]
Entrdas: [0 1] Salidas: [0.96251821]
Entrdas: [1 0] Salidas: [0.9606063]
Entrdas: [1 1] Salidas: [0.04132114]


In [17]:


nn.print_weights()



LISTADO PESOS DE CONEXIONES
[[ 0.33312049  0.96929334 -1.01181827]
 [-1.1970704  -0.56639392  2.55493383]
 [-0.20677344 -1.19665857  2.50590589]]
[[1.46094368]
 [1.90995267]
 [2.45818313]]


In [14]:
# funcion Coche Evita obstáculos
nn = NeuralNetwork([2,3,2],activation ='tanh')
X = np.array([[0, 0],   # sin obstaculos
              [0, 1],   # sin obstaculos
              [0, -1],  # sin obstaculos
              [0.5, 1], # obstaculo detectado a derecha
              [0.5,-1], # obstaculo a izq
              [1,1],    # demasiado cerca a derecha
              [1,-1]])  # demasiado cerca a izq

y = np.array([[0,1],    # avanzar
              [0,1],    # avanzar
              [0,1],    # avanzar
              [-1,1],   # giro izquierda
              [1,1],    # giro derecha
              [0,-1],   # retroceder
              [0,-1]])  # retroceder
nn.fit(X, y, learning_rate=0.03,epochs=15001)

index=0
for e in X:
    print("X:",e,"y:",y[index],"Network:",nn.predict(e))
    index=index+1

epochs: 0
epochs: 10000
X: [0. 0.] y: [0 1] Network: [0.00191208 0.999987  ]
X: [0. 1.] y: [0 1] Network: [-0.00155286  0.99998227]
X: [ 0. -1.] y: [0 1] Network: [-5.56800735e-04  9.99923778e-01]
X: [0.5 1. ] y: [-1  1] Network: [-0.9361485   0.95287172]
X: [ 0.5 -1. ] y: [1 1] Network: [0.95959703 0.95602061]
X: [1. 1.] y: [ 0 -1] Network: [-0.00893906 -0.96409455]
X: [ 1. -1.] y: [ 0 -1] Network: [ 0.00355377 -0.97053715]


In [15]:


nn.print_weights()



LISTADO PESOS DE CONEXIONES
[[ 0.47347102 -1.72279585 -1.52805261 -1.43949019]
 [-0.65631183  3.1023058   3.17519661 -0.29291389]
 [ 1.14745727  0.67175474 -0.97223917 -0.18915629]]
[[ 0.89638539  0.16792411]
 [-1.75553314 -2.79733823]
 [ 2.05130749 -1.8731057 ]
 [ 0.19422289 -1.75548772]]
